In [1]:
import sys
#sys.path.append('../HoVer-UNet/')
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(pow(2,40))
import pickle
from glob import glob
from typing import Tuple
from skimage.io import imread
import cv2
import random
import numpy as np
import segmentation_models_pytorch as smp
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
from PIL import Image
#from patchify import patchify, unpatchify
from torch.utils.data import Dataset, DataLoader
import glob
from tqdm.notebook import tqdm
import openslide
import pandas as pd
from models.HoVerNet.post_proc import process
import time
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.cuda.amp import autocast, GradScaler
from tensorboardX import SummaryWriter
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import json
import random
from sklearn.metrics import roc_auc_score
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import glob
import shutil
import openslide
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import gc

### Replace with your own WSI filepath and save dir

In [2]:
input_wsi_path = '../GTEX-1117F-0226.svs'
save_nuc_map_path = '../nulci.png'

In [3]:
label_map = {
    'CC': 'CC',
    'EmAC': 'EC',
    'MC': 'MC',
    'PSPC': 'SC',
    'PsC': 'SC',
    'PSC': 'SC',
    'Psc': 'SC',
    'PSPC': 'SC',
    'UC': 'Other'
}

### Download hover_unet_weights.pth and place it in this folder

In [4]:
fasthovernet = smp.Unet(encoder_name='mit_b2', classes=10, in_channels=3, encoder_weights=None).cuda()
fasthovernet.load_state_dict(torch.load('../model_weights/hover_unet_weights.pth')['model_state_dict'])
_ = fasthovernet.eval()

In [5]:
slide = openslide.OpenSlide(input_wsi_path)
level = slide.properties.get('openslide.objective-power')

width, height = slide.level_dimensions[0]
im_20x = slide.read_region((0, 0), 0, (width, height))
im_20x = np.array(im_20x)[:, :, :3]

if level == '40':
    im_20x = cv2.resize(im_20x, (im_20x.shape[1]//2, im_20x.shape[0]//2))
elif level == '20':
    pass
else:
    raise ValueError('Only WSIs with magnifications of 20 and 40x are supported currently.')
    
h, w, c = im_20x.shape

patch_size = 512
stride = 256

full_pred_map = torch.zeros((10, h, w), dtype=torch.float16)
full_overlap_map = torch.zeros((h, w), dtype=torch.int8)

for x_start in tqdm(range(0, w-patch_size, stride)):
    for y_start in range(0, h-patch_size, stride):

        ipt = torch.tensor(im_20x[y_start:y_start+patch_size, x_start:x_start+patch_size]).unsqueeze(0).permute(0, 3, 1, 2) / 255
        with torch.no_grad():
            preds = fasthovernet(ipt.cuda())

        full_pred_map[:, y_start:y_start+patch_size, x_start:x_start+patch_size] += preds[0].cpu()
        full_overlap_map[y_start:y_start+patch_size, x_start:x_start+patch_size] += 1

del im_20x
gc.collect()

preds = full_pred_map / full_overlap_map

preds = preds.permute(1, 2, 0).unsqueeze(0).float()

del full_pred_map, full_overlap_map
gc.collect()

pred_np = F.softmax(preds[..., :2], dim=-1)[..., 1][..., None]
pred_hv = preds[..., 2:4]
pred_tp = torch.argmax(F.softmax(preds[..., 4:], dim=-1), dim=-1)[..., None]
preds = torch.cat([pred_tp, pred_np, pred_hv], dim=-1)

mask = preds[:, :, :, 0].cpu().numpy()#.astype(np.uint8)
binary_mask = (mask[0] > 0).astype(np.uint8)*255

del mask, preds, pred_tp, pred_hv, pred_np
gc.collect()

cv2.imwrite(save_nuc_map_path, binary_mask)

del binary_mask
gc.collect()

  0%|          | 0/185 [00:00<?, ?it/s]

0